> [!TIP]
> When?
> 
> ***Pluggable Object*** is used to avoid ugly duplicated conditional (duplicated if-else).

Example: When writing a graphics editor,
selection is actually a bit complicated. If you’re over a figure when you press the
mouse button, then subsequent moves of the mouse move that figure and
releasing the mouse button leaves the figure selected. If you’re not over a figure,
then you are selecting a group of figures, and subsequent moves of the mouse
typically resize a rectangle used to select several figures. Releasing the mouse
button causes the figures inside the rectangle to be selected.

The initial code looks like this:

```python
self.selected_figure = None

def mouse_down(self):
    if self.is_over_a_figure():
        self.selected_figure = self.find_figure()

def mouse_move(self):
    if self.selected_figure is not None:
        self.move_figure()
    else:
        self.resize_selection_rectangle()

def mouse_up(self):
    if self.selected_figure is not None:    # duplicated specific condition with mouse_move
        self.selected_figure = None
    else:   # duplicated specific condition in mouse_move
        self.select_figures_in_rectangle()
```

**Problem**: condition is likely to spread. Because, different conditions in `mouse_move` requires different behavior in `mouse_up`, resulting in inevitable different conditions in `mouse_up` to handle individual cases in `mouse_move`.

**Solution**: create 2 pluggable objects:
* `SingleSelection` to behave (mouse move and mouse up) correctly when the mouse was pressed over a figure.
* `MultipleSelection` to behave (mouse move and mouse up) correctly when the mouse was pressed over a blank area.

```python
self.selection_mode = None: SelectionMode

def mouse_down(self):
    if self.is_over_a_figure():
        self.selection_mode = SingleSelection(self)
    else:
        self.selection_mode = MultipleSelection(self)

def mouse_move(self):
    self.selection_mode.mouse_move()

def mouse_up(self):
    self.selection_mode.mouse_up()
```

The `SingleSelection` and `MultipleSelection` are must implement the same interface defined by `SelectionMode`.

> [!TIP]
> How?
>
> Replaced if-else with polymorphism. 

The code is now more readable and maintainable.